In [8]:
import pandas as pd
import sklearn
import numpy as np

In [9]:
df = pd.read_csv("../Binance/btcusdt.csv")
df

,Unnamed: 0,Open,Close,High,Low
0,0,739,736,739,736
1,1,736,742,742,736
2,2,742,736,742,736
3,3,736,733,736,733
4,4,733,730,733,730
...,...,...,...,...,...
323743,323743,1522,1528,1528,1522
323744,323744,1528,1531,1531,1528
323745,323745,1531,1534,1534,1531
323746,323746,1534,1528,1534,1528


In [10]:
# #Check if any zero volumes are available
# indexZeros = df[ df['Volume'] == 0 ].index

# df.drop(indexZeros , inplace=True)
# df.loc[(df["Volume"] == 0 )]
# df.isna().sum()

In [11]:
df.head()

,Unnamed: 0,Open,Close,High,Low
0,0,739,736,739,736
1,1,736,742,742,736
2,2,742,736,742,736
3,3,736,733,736,733
4,4,733,730,733,730


In [12]:
import pandas_ta as ta

In [13]:
df['ATR'] = df.ta.atr(length=20)
df['RSI'] = df.ta.rsi()
df['EMA13'] = df.ta.ema(length=13)
df['EMA21'] = df.ta.ema(length=21)

from scipy.stats import linregress
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

In [14]:
backrollingN = 8

In [15]:
df['slopeRSI'] = df['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA13'] = df['EMA13'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeEMA21'] = df['EMA21'].rolling(window=backrollingN).apply(get_slope, raw=True)

In [16]:
df = df.dropna()

In [17]:
df

,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA13,EMA21,slopeRSI,slopeEMA13,slopeEMA21
27,27,733,736,736,733,4.028555,52.775893,731.037162,729.913725,1.248667,-0.035786,0.081869
28,28,736,739,739,736,3.977127,55.298270,732.174711,730.739750,1.842522,0.219327,0.233517
29,29,739,742,742,739,3.928270,57.729724,733.578323,731.763409,2.130493,0.504321,0.413587
30,30,742,745,745,742,3.881857,60.068771,735.209992,732.966735,1.997949,0.753703,0.580534
31,31,745,748,748,745,3.837764,62.314527,737.037136,734.333396,2.276412,1.028685,0.771264
...,...,...,...,...,...,...,...,...,...,...,...,...
323743,323743,1522,1528,1528,1522,3.822366,51.937356,1524.651092,1525.545370,2.320200,-0.190900,-0.262035
323744,323744,1528,1531,1531,1528,3.781248,54.632721,1525.558079,1526.041246,2.297104,0.132290,-0.049902
323745,323745,1531,1534,1534,1531,3.742186,57.216588,1526.764067,1526.764769,1.824616,0.373595,0.120219
323746,323746,1534,1528,1534,1528,3.855076,50.964696,1526.940629,1526.877062,1.108262,0.493694,0.219680


In [18]:
#Target flexible way
pipdiff = 500*1e-5 #for TP
SLTPRatio = 2 #pipdiff/Ratio gives SL

def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open = list(df1['Open'])
    trendcat = [None] * length
    
    for line in range (0,length-barsupfront-2):
        valueOpenLow = 0
        valueOpenHigh = 0
        for i in range(1,barsupfront+2):
            value1 = open[line+1]-low[line+i]
            value2 = open[line+1]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= pipdiff) and (-valueOpenHigh <= (pipdiff/SLTPRatio)) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ( (valueOpenLow <= (pipdiff/SLTPRatio)) and (-valueOpenHigh >= pipdiff) ):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

In [19]:
# mytarget(barsfront to take into account, dataframe)
df['mytarget'] = mytarget(16, df)
df.tail()

C:\Users\hites\AppData\Local\Temp/ipykernel_4040/2438764458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mytarget'] = mytarget(16, df)


,Unnamed: 0,Open,Close,High,Low,ATR,RSI,EMA13,EMA21,slopeRSI,slopeEMA13,slopeEMA21,mytarget
323743,323743,1522,1528,1528,1522,3.822366,51.937356,1524.651092,1525.545370,2.320200,-0.190900,-0.262035,NaN
323744,323744,1528,1531,1531,1528,3.781248,54.632721,1525.558079,1526.041246,2.297104,0.132290,-0.049902,NaN
323745,323745,1531,1534,1534,1531,3.742186,57.216588,1526.764067,1526.764769,1.824616,0.373595,0.120219,NaN
323746,323746,1534,1528,1534,1528,3.855076,50.964696,1526.940629,1526.877062,1.108262,0.493694,0.219680,NaN
323747,323747,1528,1525,1528,1525,3.812323,48.132751,1526.663396,1526.706420,0.312733,0.489493,0.241917,NaN


In [21]:
df = df.dropna()
attributes = ['ATR', 'RSI', 'EMA13', 'slopeRSI', 'slopeEMA13']
x = df[attributes]
y = df['mytarget']
print(x)

             ATR        RSI        EMA13  slopeRSI  slopeEMA13
27      4.028555  52.775893   731.037162  1.248667   -0.035786
28      3.977127  55.298270   732.174711  1.842522    0.219327
29      3.928270  57.729724   733.578323  2.130493    0.504321
30      3.881857  60.068771   735.209992  1.997949    0.753703
31      3.837764  62.314527   737.037136  2.276412    1.028685
...          ...        ...          ...       ...         ...
323725  3.681695  51.744363  1531.928076 -0.717842    1.261997
323726  3.647610  48.680999  1531.366923 -1.828530    0.918447
323727  3.765230  54.484731  1531.743076 -2.003392    0.583158
323728  3.726968  57.097304  1532.494066 -1.598941    0.336585
323729  3.690620  59.594954  1533.566342 -0.641843    0.247685

[323703 rows x 5 columns]


In [22]:
df.isna().sum()

Unnamed: 0    0
Open          0
Close         0
High          0
Low           0
ATR           0
RSI           0
EMA13         0
EMA21         0
slopeRSI      0
slopeEMA13    0
slopeEMA21    0
mytarget      0
dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=2)

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
model = DecisionTreeClassifier()

In [26]:
model.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [27]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 100% 
accuracy test 63% 


In [28]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
# accuracy_tesst = accuracy_score(ytest, pred_test)
# print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    50.041242
1.0    49.958758
Name: mytarget, dtype: float64


In [29]:
train_index = int(0.8 * len(x))
xtrain, xtest = x[:train_index], x[train_index:]
ytrain, ytest = y[:train_index], y[train_index:]

In [30]:
model2 = DecisionTreeClassifier()
model2.fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)

In [31]:
from sklearn.metrics import accuracy_score
accurancy_train = accuracy_score(ytrain, y_pred_train)
accuracy_tesst = accuracy_score(ytest, y_pred_test)

print("accuracy train %.f%% "% (accurancy_train*100))
print("accuracy test %.f%% "% (accuracy_tesst*100))

accuracy train 93% 
accuracy test 92% 


In [32]:
print(df['mytarget'].value_counts()*100/df['mytarget'].count())


pred_test = np.random.choice([0,1,2], len(y_pred_test))
accuracy_tesst = accuracy_score(ytest, pred_test)
print("accuracy gambler %.2f%%" %(accuracy_tesst*100))

2.0    50.041242
1.0    49.958758
Name: mytarget, dtype: float64
accuracy gambler 33.71%


In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd



# Create the contingency mat
matrix_test = confusion_matrix(ytest, y_pred_test)
matrix_train = confusion_matrix(ytrain, y_pred_train)
classification_report_pred = classification_report(ytest, y_pred_test, target_names=["downtrend", "uptrend"])

# Convert the matrix to a DataFrame for easy display

matrix_df_test = pd.DataFrame(matrix_test, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])
# matrix_df_test = pd.DataFrame(matrix_test, columns=['pred_sideways','pred_Downtrend','pred_Uptrend'], index=['Sidetrend','Downtrend','Uptrend'])

matrix_df_train = pd.DataFrame(matrix_train, columns=['pred_Downtrend','pred_Uptrend'], index=['Downtrend','Uptrend'])
# matrix_df_train = pd.DataFrame(matrix_train, columns=['pred_sideways','pred_Downtrend','pred_Uptrend'], index=['Sidetrend','Downtrend','Uptrend'])
print("Test-Set Confusion Matrix\n")
print(matrix_df_test)
print("-----------------------------------------------------------------------------")
print("Train-Set Confusion Matrix\n")
print(matrix_df_train)
print("-----------------------------------------------------------------------------")
print("Classification Report\n")
print(classification_report_pred)

Test-Set Confusion Matrix

           pred_Downtrend  pred_Uptrend
Downtrend           29995          2599
Uptrend              2574         29573
-----------------------------------------------------------------------------
Train-Set Confusion Matrix

           pred_Downtrend  pred_Uptrend
Downtrend          119924          9200
Uptrend              9491        120347
-----------------------------------------------------------------------------
Classification Report

              precision    recall  f1-score   support

   downtrend       0.92      0.92      0.92     32594
     uptrend       0.92      0.92      0.92     32147

    accuracy                           0.92     64741
   macro avg       0.92      0.92      0.92     64741
weighted avg       0.92      0.92      0.92     64741



In [34]:
200000

200000

In [35]:
800000

800000